<h1 style="font-family:Times New Roman;color:black">Парсер стоимости ГСМ</h1>

*Nikolay Sitala 2021, nsitala@ukr.net* 
>Данные по материалам сайта www.minfin.com.ua

<h2 style="font-family: 'Times New Roman'">Выберите год</h2>

In [22]:
import ipywidgets as widgets
import datetime

now = datetime.datetime.now()

cur_year = now.year
cur_month = now.month

val_year = [cur_year - b for b in range (0, 6)]

item_year = widgets.Select(
    options=val_year,
    value=val_year[0],
    # rows=10,
    description='Год:',
    disabled=False
)
item_year

Select(description='Год:', options=(2021, 2020, 2019, 2018, 2017, 2016), value=2021)

<h2 style="font-family: 'Times New Roman'">Выберите месяц</h2>

In [23]:
name_of_months = []
dict_of_months = {1: 'Январь', 2:'Февраль', 3:'Март', 4:'Апрель', 5:'Май', 6:'Июнь', 
                  7:'Июль', 8:'Август', 9:'Сентябрь', 10:'Октябрь', 11:'Ноябрь', 12:'Декабрь'}
val_month = dict_of_months[1]

for b, v in dict_of_months.items():
    name_of_months.append(v)
    if cur_month == b:
        val_month = v

item_month = widgets.Select(
    options=name_of_months,
    value=val_month,
    rows=10,
    description='Месяц:',
    disabled=False
)
item_month

Select(description='Месяц:', index=6, options=('Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', '…

<h2 style="font-family: 'Times New Roman'">Выделите необходимые виды ГСМ через Ctrl</h2>

In [24]:
dict_name_fuel = {'dt': 'Дт', 'a92': 'A92', 'a95': 'A95', 
                  'a96': 'A95 Premium', 'lpg': 'Газ'}
list_of_name_fuel = [v for b, v in dict_name_fuel.items()]

prog_fuel = widgets.SelectMultiple(
    options=list_of_name_fuel,
    value=['Дт'],
    rows=5,
    description='ГСМ',
    disabled=False
)

prog_fuel

SelectMultiple(description='ГСМ', index=(0,), options=('Дт', 'A92', 'A95', 'A95 Premium', 'Газ'), value=('Дт',…

<h2 style="font-family: 'Times New Roman'">Скрипт парсера ГСМ</h2>

In [27]:
from typing import Any

import re
import requests
from bs4 import BeautifulSoup

import ipysheet

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

idx = 0

count_rows_fuel = len(prog_fuel.value) * 31
sheet = ipysheet.sheet(rows=count_rows_fuel, columns=5)
sheet.column_headers = ['Дата', 'Топливо', 'Средняя стоимость грн', 'Год', 'Месяц']


list_of_query_fuel = []
set_fuel = set(prog_fuel.value)

for b, v in dict_name_fuel.items():
    if v in set_fuel:
        list_of_query_fuel.append(b)

for b, v in dict_of_months.items():
    if item_month.value == v:
        number_month = b
        break
        
if number_month < 10:
    val_str_month = ''.join(['0', str(number_month)])
else:
    val_str_month = str(number_month)


def additional_cells(date_fuel, name_oil, big):
    
    global val_year, val_month
    global sheet, idx
    
    ipysheet.cell(idx, 0, date_fuel)
    ipysheet.cell(idx, 1, name_oil)
    ipysheet.cell(idx, 2, big)
    ipysheet.cell(idx, 3, val_year[0])
    ipysheet.cell(idx, 4, val_month)
    idx = idx + 1
    

def get_index(period, oil) -> Any:
    baseURL = f"https://index.minfin.com.ua/markets/fuel/{oil}/{period}/"
    r = requests.get(baseURL, headers=headers)
    return BeautifulSoup(r.content, 'html.parser')


def get_data_table(data_table, qcls, name_oil) -> None:

    date_fuel = None

    for b in data_table:
#         q = b.contents

        if b['class'][0] == qcls and b['align'] == 'left':
            date_fuel = str(b.contents[0])

        for tag in b.contents:
            try:
                big = tag.contents[0]
                if tag.name == 'big':
#                     print(idx, date_fuel, name_oil, big, sep='|')
                    additional_cells(date_fuel, name_oil, big)
            except Exception:
                pass


def start_parser_minfin():

    global item_year

    query_fuels = list_of_query_fuel
    periods = ['-'.join([str(item_year.value), val_str_month]), ]
    
    for type_fuel in query_fuels:

        name_fuel = dict_name_fuel[type_fuel]

        for chunk in periods:

            soup1 = get_index(chunk, type_fuel)

            for string in soup1.find_all(class_=re.compile("^zebra")):
                string0 = string.find_all(class_=re.compile("^r0"))
                string1 = string.find_all(class_=re.compile("^r1"))

                get_data_table(string0, 'r0', name_fuel)
                get_data_table(string1, 'r1', name_fuel)


if __name__ == "__main__":
    start_parser_minfin()

<h2 style="font-family: 'Times New Roman'">Отобразить данные в таблице</h2>

>Для копирование выделите мышью столбцы и скопируйте через Ctrl+C

In [28]:
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='01.07.2021 '), Cel…